In [1]:
import pandas as pd
import os

In [36]:
df = pd.read_csv('/Users/carboni/Downloads/Leopard Dec 19 2023.csv')

### Clusters and other keywords

In [37]:
#keyword = 'Bête'
#df = df[df['cluster'].str.contains(keyword, case=False, na=False)]

In [38]:
df.head()

,cluster,manifest_url,canvas_number,image_url,City,Country,Title,wkt,Date,Journal Type,notice
0,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,11.0,https://digi.ub.uni-heidelberg.de/iiif/2/punch...,London,United Kingdom,Punch's Almanack,POINT(-0.1275 51.507222222222),1921-01-01,Humour,NaN
1,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,12.0,https://digi.ub.uni-heidelberg.de/iiif/2/fb164...,Munich,Germany,Fliegende Blätter,POINT(11.575 48.1375),1926-01-01,Humour,NaN
2,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,13.0,https://digi.ub.uni-heidelberg.de/iiif/2/studi...,London,United Kingdom,Studio : international art,POINT(-0.1275 51.507222222222),1901-01-01,Modern Art Journal,NaN
3,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,33.0,https://digi.ub.uni-heidelberg.de/iiif/2/punch...,London,United Kingdom,Punch,POINT(-0.1275 51.507222222222),1924-01-01,Humour,NaN
4,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,34.0,https://digi.ub.uni-heidelberg.de/iiif/2/punch...,London,United Kingdom,Punch,POINT(-0.1275 51.507222222222),1924-01-01,Humour,NaN


## Index city 

### If WKT

In [39]:
df[['lon', 'lat']] = df['wkt'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

### if not wkt

In [40]:
city_coords = df.groupby('City').agg({'lat': 'first', 'lon': 'first'}).reset_index()

In [41]:
city_coords.rename({'City': 'name'}, axis=1, inplace=True)

In [42]:
city_coords.index = city_coords.index + 1
city_coords.index.name = 'id'

In [43]:
print(city_coords)

                 name                lat                  lon
id                                                           
1            Aberdeen  46.97536111111111  -123.81572222222222
2         Albuquerque    35.116666666667     -106.61666666667
3             Algiers    36.776388888889      3.0586111111111
4                Alma    43.378888888889     -84.659722222222
5            Amarillo    35.199166666667     -101.84527777778
..                ...                ...                  ...
222        Wilmington    34.223333333333     -77.912222222222
223  Wisconsin Rapids    44.394444444444     -89.832777777778
224              Yale    43.130277777778     -82.797777777778
225            Évreux    49.023333333333               1.1525
226              Žiri          46.046019            14.108283

[226 rows x 3 columns]


In [44]:
city_coords.to_csv('od_city_index.csv', index=True)

## Index to OD Matrix

In [45]:
#df['Year'] = df['date'].str.split('-', n=1, expand=True)[0]

In [46]:
df['Year'] = pd.to_datetime(df['Date']).dt.year

In [47]:
df.head()

,cluster,manifest_url,canvas_number,image_url,City,Country,Title,wkt,Date,Journal Type,notice,lon,lat,Year
0,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,11.0,https://digi.ub.uni-heidelberg.de/iiif/2/punch...,London,United Kingdom,Punch's Almanack,POINT(-0.1275 51.507222222222),1921-01-01,Humour,NaN,-0.1275,51.507222222222,1921
1,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,12.0,https://digi.ub.uni-heidelberg.de/iiif/2/fb164...,Munich,Germany,Fliegende Blätter,POINT(11.575 48.1375),1926-01-01,Humour,NaN,11.575,48.1375,1926
2,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,13.0,https://digi.ub.uni-heidelberg.de/iiif/2/studi...,London,United Kingdom,Studio : international art,POINT(-0.1275 51.507222222222),1901-01-01,Modern Art Journal,NaN,-0.1275,51.507222222222,1901
3,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,33.0,https://digi.ub.uni-heidelberg.de/iiif/2/punch...,London,United Kingdom,Punch,POINT(-0.1275 51.507222222222),1924-01-01,Humour,NaN,-0.1275,51.507222222222,1924
4,Léopard Fourrure,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,34.0,https://digi.ub.uni-heidelberg.de/iiif/2/punch...,London,United Kingdom,Punch,POINT(-0.1275 51.507222222222),1924-01-01,Humour,NaN,-0.1275,51.507222222222,1924


In [48]:
city_id_map = city_coords.reset_index().set_index('name')['id'].to_dict()

In [50]:
df = df.dropna(subset=['City'])

In [51]:
df['origin'] = df['City'].map(city_id_map).astype(int)
df['dest'] = df['City'].shift(-1).map(city_id_map).fillna(method='ffill').astype(int)
df.iloc[-1, df.columns.get_loc('dest')] = df.iloc[-1]['origin']

### Overall OD Matrix

In [52]:
overall_od_matrix = df.groupby(['origin', 'dest', 'Year']).size().reset_index(name='count')

In [53]:
overall_od_matrix['time'] = overall_od_matrix['Year'].astype(str) + '-01-01'
overall_od_matrix.drop('Year', axis=1, inplace=True)

In [54]:
print(overall_od_matrix)

      origin  dest  count        time
0          1   137      1  1909-01-01
1          1   146      1  1908-01-01
2          2    45      1  1906-01-01
3          2    54      1  1913-01-01
4          2   146      1  1915-01-01
...      ...   ...    ...         ...
1699     222   186      1  1921-01-01
1700     223    43      1  1909-01-01
1701     224   217      1  1892-01-01
1702     225   157      1  1898-01-01
1703     226    67      1  1997-01-01

[1704 rows x 4 columns]


In [55]:
overall_od_matrix.to_csv('overall_od_matrix.csv', index=False)